In [1]:
!pip install datasets
!pip install transformers
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
/bin/bash: line 1: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using 

In [2]:
!pip install pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 4.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
# from DLAIUtils import Utils
# import DLAIUtils

import os
import time
import torch

from tqdm.auto import tqdm

In [4]:
dataset = load_dataset('quora', split='train[240000:290000]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

In [5]:
dataset[0]

{'questions': {'id': [207550, 351729],
  'text': ['What is the truth of life?', "What's the evil truth of life?"]},
 'is_duplicate': False}

In [6]:
questions = []
for record in dataset:
  # questions.extend(record.questions['text'])
  questions.extend(record['questions']['text'])

question = list(set(questions))



In [7]:
len(question)

88919

In [8]:
question[9]

'How can I start a music band?'

In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
query = "which city is most populated in the world?"
xq = model.encode(query)


In [11]:
xq.shape

(384,)

In [12]:
PINECONE_API_KEY = ""

In [13]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)


In [ ]:
pinecone.create_index(name="quora-index",
    dimension=model.get_sentence_embedding_dimension(),
    metric='cosine',
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) )

In [14]:
index = pinecone.Index("quora-index")

In [15]:
index

In [16]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
  i_end = min(i+batch_size, len(questions))
  ids = [str(x) for x in range(i, i_end)]
  metadatas = [{'text': text} for text in questions[i:i_end]]
  xc = model.encode(questions[i:i_end])
  records = zip(ids, xc, metadatas)
  index.upsert(vectors=records)

  0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000}

In [18]:
embedding = model.encode(query).tolist()
results = index.query(top_k=10,vector=embedding,include_metadata=True,include_values=False)
results

{'matches': [{'id': '4755',
              'metadata': {'text': 'Where is the most beautiful city in the '
                                   'world?'},
              'score': 0.738103271,
              'values': []},
             {'id': '4608',
              'metadata': {'text': "How is the world's population estimated?"},
              'score': 0.595202327,
              'values': []},
             {'id': '2979',
              'metadata': {'text': 'What country has the fastest growing '
                                   'population and why?'},
              'score': 0.592233,
              'values': []},
             {'id': '9609',
              'metadata': {'text': "Which country's women are most beautiful "
                                   'in the world?'},
              'score': 0.517084837,
              'values': []},
             {'id': '116',
              'metadata': {'text': 'Where are the largest slums in the world?'},
              'score': 0.516438,
              'value